In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
sns.set()

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)
    except Exception as e:
        return pd.read_csv(str(path), *args, **kwargs)

def followorbit(vals, path):
    out = []
    for n in range(100):
        closest05 = np.clip(round(path[n] * 2) / 2, 0, 10)
        rank = vals.query(f'nbopp == {n} and othercoop == {closest05}')['rank'].values[0]
        out.append(rank)
    return np.array(out)

import re
def gethighest(vals, prefix="logall_", which=1):
    if which == 1:
        m = np.argmax([int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals])
    else:
        trueval = [int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals]
        itrueval = sorted(list(enumerate(trueval)), key=lambda x: x[1])
        m = itrueval[-which][0]
    return vals[m]

    
def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

import platform
hostname = platform.node()
print(hostname)

if hostname == 'cluster':
    prefix = '/home/ecoffet/robocoop/'
else:
    prefix = '/home/pecoffet/remoterobo/'

In [ ]:
paths = sorted(glob.glob(f'{prefix}/logs/negociate4/**/rep00/', recursive=True))

for path in paths:
    if 'train' in path:
        continue
    print(path)
    try:
        fitness = read_csvx(path+'../fit.txt', delimiter="\t", names=['gen', 'min', 'q1', 'med', 'q3', 'max'])
        fitness['med'].plot()
        plt.show()
        try:
            print(gethighest(glob.glob(path + '/coop*'), 'coop_'))
            coop = read_csvx(gethighest(glob.glob(path + '/coop*'), 'coop_'), delimiter="\t")
            sns.distplot(coop['coop'])
            plt.xlim(0, 10)
            plt.show()
        except Exception as e:
            print(e)
        try:
            print(gethighest(glob.glob(path + '/analysis*'), 'analysis_log_', 2))
            choice = read_csvx(gethighest(glob.glob(path + '/analysis*'), 'analysis_log_', 2), delimiter="\t")
            choice['other'] = np.round(choice['other'], 2)
            choice['owncoop'] = np.round(choice['owncoop'], 2)
            sns.heatmap(pd.pivot_table(choice, 'accept', 'owncoop', 'other'))
            plt.show()
        except Exception as e:
            print(e)
        print(gethighest(glob.glob(path + '/analysis*'), 'analysis_log_'))
        choice = read_csvx(gethighest(glob.glob(path + '/analysis*'), 'analysis_log_'), delimiter="\t")
        choice['other'] = np.round(choice['other'], 2)
        choice['owncoop'] = np.round(choice['owncoop'], 2)
        sns.heatmap(pd.pivot_table(choice, 'accept', 'owncoop', 'other'))
        plt.show()

    except Exception as e:
        print(e)